In [2]:
import numpy as np
import copy

In [1]:
import Utility.notebookutil as nbu
import sys
import importlib
sys.meta_path.append(nbu.NotebookFinder())

In [14]:
import PCA as pca

In [5]:
#make [id, x, y, cluster id] data with cluster id = 0 to initialize the data
def initialize_xy_data(xy_data):
    initial_data = []
    for i in xy_data:
        initial_data.append([i[0], i[1], i[2], 0])
    return initial_data

In [18]:
def add_new_cluster(base_data, target_cluster, split_index, split_dim, new_cluster_index, 
                    th = 10):
    new_cluster_data = []
    cluster_points_list = []
    for i in range(new_cluster_index):
        cluster_points_list.append([])
    
    for point in base_data:
        index = int(point[-1])
        cluster_points_list[index].append(point)
    
    #target_cluster or new cluster do not have enough spectra 
    if(len(cluster_points_list[target_cluster]) < split_index + th or split_index < th):
        return 0, False, 0
    
    #invalid split dimension
    #split_dim = 0 -> split using x axis
    #split_dim = 1 -> split using y axis
    if(split_dim > 1):
        print "error: invalid split dimension: " + str(split_dim) 
        return 0, False, 0
    
    split_points_list = np.array(cluster_points_list[target_cluster])[:, 1+split_dim]
    split_value = sorted(split_points_list)[split_index]
    
    for point in base_data:
        new_data = copy.deepcopy(point)
        if(new_data[-1] == target_cluster):
            if(new_data[1 + split_dim] >= split_value):
                new_data[-1] = new_cluster_index
        new_cluster_data.append(new_data)
    return np.array(new_cluster_data), True, split_value

In [10]:
def prepare_cluster_table(base_data, split_data, new_cluster_index):
    new_data = []
    if(len(split_data) != 3):
        print "bad splitList " + split_data
        return -1
    split_value = split_data[0]
    split_dim = split_data[1]
    split_target = split_data[2]

    #point has [x, y, cluster index]
    for point in base_data: 
        if(point[3] == split_target):
            if(point[1 + split_dim] >= split_value):
                point[3] = new_cluster_index
        new_data.append(point)
    return np.array(new_data)

In [17]:
def split_and_clustering(xy_data, spectra, target_cluster, split_index, split_dim, new_cluster_index, current_tree):
    new_xy_data, success_flag, split_value = add_new_cluster(xy_data, target_cluster, split_index,
                                                                 split_dim, new_cluster_index)
    if(success_flag == 1):
        split_key = [split_value, split_dim, target_cluster]
        new_tree = current_tree + [split_key]
        pca.pca_for_clusters(new_xy_data, spectra, new_cluster_index + 1)
        return new_tree
    return False            